<a href="https://colab.research.google.com/github/hariprasadpl088/Peft-Lora/blob/main/Peft_Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.3 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer,TrainingArguments,Trainer
from peft import get_peft_model,LoraConfig,TaskType
from datasets import Dataset

In [ ]:
data={
    "text": [
        "### Question: Who is the goat of football?\n### Answer: The goat of football is Lionel Andres Messi."
        "### Question: Who won the champions league trophy in 2025?\n### Answer: PSG won the champions league in 2025."
        "### Question: Which is the best football club in the world?\n### Answer: The one and only answer is Barcelona."
        "### Question: who will be the winner of the club world cup in 2025?\n### Answer: PSG has the chance to win the club world cup."
    ]
}
dataset=Dataset.from_dict(data)

In [ ]:
model_name="tiiuae/falcon-rw-1b"
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token=tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [ ]:
def tokenize(batch):
  tokens=tokenizer(
      batch["text"],
      padding="max_length",
      truncation=True,
      max_length=128
  )
  tokens["labels"]=tokens["input_ids"].copy()
  return tokens

In [ ]:
tokenized_dataset=dataset.map(tokenize,batched=True)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto"
)

config.json: 0.00B [00:00, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


pytorch_model.bin:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [ ]:
lora_config=LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [ ]:
model=get_peft_model(model,lora_config)
model.print_trainable_parameters()

trainable params: 1,572,864 || all params: 1,313,198,080 || trainable%: 0.1198


In [ ]:
training_args=TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    learning_rate=2e-4,
    logging_steps=1,
    save_strategy="no",
    report_to="none"
)

In [ ]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)
trainer.train()

/tmp/ipython-input-14-1041140387.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,2.748700
2,2.600300
3,2.490300


TrainOutput(global_step=3, training_loss=2.613084157307943, metrics={'train_runtime': 3.5648, 'train_samples_per_second': 0.842, 'train_steps_per_second': 0.842, 'total_flos': 2788244324352.0, 'train_loss': 2.613084157307943, 'epoch': 3.0})

In [ ]:
model.save_pretrained("lora-finetuned")
tokenizer.save_pretrained("lora-finetuned")

('lora-finetuned/tokenizer_config.json',
 'lora-finetuned/special_tokens_map.json',
 'lora-finetuned/vocab.json',
 'lora-finetuned/merges.txt',
 'lora-finetuned/added_tokens.json',
 'lora-finetuned/tokenizer.json')